In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10344, done.
remote: Total 10344 (delta 0), reused 0 (delta 0), pack-reused 10344
Receiving objects: 100% (10344/10344), 10.58 MiB | 14.66 MiB/s, done.
Resolving deltas: 100% (7141/7141), done.


In [3]:
!mv yolov5/* ./

In [2]:
df = pd.read_csv('input/train.csv')
df

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]
...,...,...,...,...,...,...
23496,2,29859,10755,2983,2-10755,[]
23497,2,29859,10756,2984,2-10756,[]
23498,2,29859,10757,2985,2-10757,[]
23499,2,29859,10758,2986,2-10758,[]


In [ ]:
import ast
from tqdm import tqdm

new_df = pd.DataFrame(columns=['image_id', 'x', 'y', 'w', 'h', 'x_center', 'y_center', 'classes'])

# idx = 4761
for idx in tqdm(range(len(df))):
    image_id, annotations = df.iloc[idx][['image_id','annotations']]
    annotations = ast.literal_eval(annotations)
    # bbox = annotations[0]
    for bbox in annotations:
        x = bbox['x']
        y = bbox['y']
        w = bbox['width']
        h = bbox['height']
        x_center = x + w/2
        y_center = y + h/2
        new_df = new_df.append({'image_id':image_id,'x':x,'y':y,'w':w,'h':h,'x_center':x_center,'y_center':y_center,'classes':0}, ignore_index=True)
new_df

In [82]:
index = list(set(new_df.image_id))

In [101]:
source = 'train_images'
import shutil as sh
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(new_df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                for mn in row:
                    mn[1] = mn[1]/1280
                    mn[3] = mn[3]/1280
                    mn[2] = mn[2]/720
                    mn[4] = mn[4]/720
                    
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            video_id, video_frame = name.split('-')[:2]
            sh.copy("input/{}/video_{}/{}.jpg".format(source,video_id,video_frame),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

100%|██████████| 4919/4919 [00:24<00:00, 202.93it/s]


'image_id'

In [ ]:
# yolov5s < yolov5m < yolov5l < yolov5x
# after test yolov5s, small batch/epoch size, apply them by your resourse
# -1 to auto batch
# you should use --img <img_size>
!python train.py --img 1280 --batch -1 --epochs 10 --data reef0.yaml --weights yolov5x.pt --name yolov5x_fold0

train: weights=yolov5x.pt, cfg=, data=reef0.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=-1, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolov5x_fold0, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 807d178 torch 1.9.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, t

In [6]:
#06번 5x epoch 1 f1 0.6정도
!python detect.py --weights runs/train/yolov5x_fold08/weights/best.pt --source convertor/fold0/images/val2017 --img 720 1280 --project runs/det

detect: weights=['runs/train/yolov5x_fold08/weights/best.pt'], source=convertor/fold0/images/val2017, imgsz=[720, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/det, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 807d178 torch 1.9.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients
image 1/983 /workspace/cutiekath/quickstarter-yolov5/convertor/fold0/images/val2017/0-1005.jpg: 736x1280 1 reef, Done. (0.165s)
image 2/983 /workspace/cutiekath/quickstarter-yolov5/convertor/fold0/images/val2017/0-1006.jpg: 736x1280 1 reef, Done. (0.138s)
image 3/983 /workspace/cutiekath/quickstarter-yolov5/convertor/fold0/images/val2017/0-101.jpg: 736x1280 2 reefs, Done. (0.138s)
image 4/983 /work

In [ ]:
hi baby